SimCLR Implementation for single GPU setup

https://medium.com/the-owl/simclr-in-pytorch-5f290cb11dd7

In [1]:
import numpy as np
import pandas as pd
import shutil, time, os, requests, random, copy

import torch
import torch.nn as nn 
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms, models

import matplotlib.pyplot as plt 

from sklearn.manifold import TSNE

In [2]:
def set_seed(seed=16):
    np.random.seed(seed)
    torch.manual_seed(seed)

In [9]:
dir_CIFAR = "/home/hoseung/Work/datasets/cifar-10-batches-py/"

## Data Loader

In [7]:
import pickle

def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [15]:
train_files = ['data_batch_1','data_batch_2','data_batch_3','data_batch_4','data_batch_5']
images = np.array([], dtype=np.uint8).reshape((0,3072))
labels = np.array([])
for tf in train_files:
    data_dict = unpickle(dir_CIFAR+tf)
    data = data_dict[b'data']
    images = np.append(images, data, axis=0)
    labels = np.append(labels, data_dict[b'labels'])


testimages = np.array([],dtype=np.uint8).reshape((0,3072))
testlabels = np.array([])

data_dict = unpickle(dir_CIFAR+'/test_batch')
data = data_dict[b'data']
testimages = np.append(testimages,data,axis=0)
testlabels = np.append(testlabels,data_dict[b'labels'])


images = images.reshape((-1,3,32,32)).astype(float)
testimages = testimages.reshape((-1,3,32,32)).astype(float)

lab_dict = {0:'airplane',1:'automobile',2:'bird',3:'cat',4:'deer',5:'dog',6:'frog',7:'horse',8:'ship',9:'truck'}

In [25]:
# split 50K images 80:20.
trimages = images[:40000]
valimages = images[40000:]
trlabels = labels[:40000]
vallabels = labels[40000:]
MEAN = np.mean(trimages/255.0,axis=(0,2,3),keepdims=True) # will be used in normalization
STD = np.std(trimages/255.0,axis=(0,2,3),keepdims=True)

## Data Generator

In [17]:
random.sample?

Signature: random.sample(population, k, *, counts=None)
Docstring:
Chooses k unique random elements from a population sequence or set.

Returns a new list containing elements from the population while
leaving the original population unchanged.  The resulting list is
in selection order so that all sub-slices will also be valid random
samples.  This allows raffle winners (the sample) to be partitioned
into grand prize and second place winners (the subslices).

Members of the population need not be hashable or unique.  If the
population contains repeats, then each occurrence is a possible
selection in the sample.

Repeated elements can be specified one at a time or with the optional
counts parameter.  For example:

    sample(['red', 'blue'], counts=[4, 2], k=5)

is equivalent to:

    sample(['red', 'red', 'red', 'red', 'blue', 'blue'], k=5)

To choose a sample from a range of integers, use range() for the
population argument.  This is especially fast and space efficient
for sampling fro

In [37]:
class C10DataGen(Dataset):
    """   
    What is **frame**?
    """
    def __init__(self,phase,imgarr,s = 0.5):
        self.phase = phase
        self.imgarr = imgarr
        self.s = s
        self.transforms = transforms.Compose([transforms.RandomHorizontalFlip(0.5),
                                              transforms.RandomResizedCrop(32,(0.8,1.0)),
                                              transforms.Compose([transforms.RandomApply([transforms.ColorJitter(0.8*self.s, 
                                                                                                                 0.8*self.s, 
                                                                                                                 0.8*self.s, 
                                                                                                                 0.2*self.s)], p = 0.8),
                                                                  transforms.RandomGrayscale(p=0.2)
                                                                 ])])
        
    def __len__(self):
        return self.imgarr.shape[0]
    
    def __getitem__(self, idx):
        x = self.imgarr[idx]
        x = x.astype(np.float32)/255.0
        
        x1 = self.augment(torch.from_numpy(x))
        x2 = self.augment(torch.from_numpy(x))
        
        x1 = self.preprocess(x1)
        x2 = self.preprocess(x2)
        
        return x1, x2
    
    def on_epoch_end(self):
        # shuffle the dataset
        #self.imgarr = self.imgarr[random.sample(list(range(self.__len__())),
        #                                        k = self.__len__())] # choose k sample from population.
        np.random.shuffle(self.imgarr) # shuffle images. -- np.random.shuffle only shuffles along the first axis.
        
    def preprocess(self, frame):
        frame = (frame-MEAN)/STD
        return frame
    
    def augment(self, frame, transformations = None):
        if self.phase == 'train':
            frame = self.transforms(frame)
        else:
            return frame
        
        return frame

Pytorch DataLoader expects a dataset (or data generator), each item of which is retrieved by `__get_item__()` or `__iter__()` depending on the type of dataset. 


In [36]:
dg = C10DataGen('train', trimages)
dl = DataLoader(dg, batch_size=128, drop_last=True)

vdg = C10DataGen('valid', valimages)
vdl = DataLoader(vdg, batch_size=128, drop_last=True)

In [ ]:
dl